In [1]:
# Import dependencies
import pandas as pd
import gmaps
import requests


# Import API key
from config import g_key

In [2]:
# Store the CSV we saved and created in part 1 into another dataframe.
city_data_df=pd.read_csv('weather_data/cities.csv')
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,Yarmouth,41.7057,-70.2286,80.15,65,0,14.97,US,2022-07-11 20:35:55
1,1,Bayeux,-7.1250,-34.9322,79.43,73,0,8.05,BR,2022-07-11 20:35:56
2,2,Lima,-12.0432,-77.0282,64.90,74,100,5.75,PE,2022-07-11 20:33:07
3,3,Petropavlovka,50.6114,105.3196,52.29,89,100,4.18,RU,2022-07-11 20:35:56
4,4,Lodja,-3.4833,23.4333,78.80,31,56,2.01,CD,2022-07-11 20:35:56


In [3]:
# Configure gmaps to use my google API key.
gmaps.configure(api_key=g_key)

In [4]:
# Get the maximum temperature
max_temp=city_data_df["Max Temp"]
temps=[]
for temp in max_temp:
    temps.append(max(temp,0))

In [11]:
# Heatmap of temperature
# Get the latitude and longitude
locations=city_data_df[["Lat","Lng"]]

# Get the maximum temperature
max_temp=city_data_df["Max Temp"]

# Assign the figure variable
fig=gmaps.figure(center=(30.0,31.0), zoom_level=1.5)

# Assign the heatmap variable
heat_layer=gmaps.heatmap_layer(locations, weights=[max(temp,0) for temp in max_temp],dissipating=False,max_intensity=300,point_radius=4)
fig = gmaps.figure()
fig.add_layer(gmaps.heatmap_layer(locations, weights=[max(temp,0) for temp in max_temp]))
fig

Figure(layout=FigureLayout(height='420px'))

In [12]:
# Heatmap of percent humidity
locations = city_data_df[["Lat", "Lng"]]
humidity = city_data_df["Humidity"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False, max_intensity=300, point_radius=4)

fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [13]:
# Heatmap of wind speeds
locations = city_data_df[["Lat", "Lng"]]
wind = city_data_df["Wind Speed"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=wind, dissipating=False, max_intensity=300, point_radius=4)

fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [14]:
# Ask the customer to add a minimum and maximum temperature value.
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [15]:
# Filter the dataset to find the cities that fit the criteria.
preferred_cities_df=city_data_df.loc[(city_data_df['Max Temp'] <= max_temp) & (city_data_df['Max Temp'] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,Yarmouth,41.7057,-70.2286,80.15,65,0,14.97,US,2022-07-11 20:35:55
1,1,Bayeux,-7.1250,-34.9322,79.43,73,0,8.05,BR,2022-07-11 20:35:56
4,4,Lodja,-3.4833,23.4333,78.80,31,56,2.01,CD,2022-07-11 20:35:56
6,6,Cayenne,4.9333,-52.3333,82.44,1,57,5.75,GF,2022-07-11 20:35:46
7,7,Avera,33.1940,-82.5271,80.65,56,97,6.11,US,2022-07-11 20:35:57
10,10,Rizhao,35.4275,119.4553,76.15,98,94,6.73,CN,2022-07-11 20:35:58
14,14,Kavaratti,10.5669,72.6420,80.89,81,100,17.56,IN,2022-07-11 20:35:59
15,15,Hilo,19.7297,-155.0900,82.27,63,40,5.99,US,2022-07-11 20:32:15
22,22,Kulhudhuffushi,6.6221,73.0700,81.52,78,100,8.41,MV,2022-07-11 20:36:00
24,24,Acapulco,16.8634,-99.8901,87.62,74,20,9.22,MX,2022-07-11 20:32:58


In [16]:
preferred_cities_df.count()

City_ID       212
City          212
Lat           212
Lng           212
Max Temp      212
Humidity      212
Cloudiness    212
Wind Speed    212
Country       212
Date          212
dtype: int64

In [22]:
# Create a dataframe called 'hotel_df' to store hotel names along with city, country, max temp and coordinates.
hotel_df=preferred_cities_df[['City','Country','Max Temp','Lat','Lng']].copy()
hotel_df['Hotel Name'] = ''
hotel_df.head(10)

,City,Country,Max Temp,Lat,Lng,Hotel Name
0,Yarmouth,US,80.15,41.7057,-70.2286,
1,Bayeux,BR,79.43,-7.1250,-34.9322,
4,Lodja,CD,78.80,-3.4833,23.4333,
6,Cayenne,GF,82.44,4.9333,-52.3333,
7,Avera,US,80.65,33.1940,-82.5271,
10,Rizhao,CN,76.15,35.4275,119.4553,
14,Kavaratti,IN,80.89,10.5669,72.6420,
15,Hilo,US,82.27,19.7297,-155.0900,
22,Kulhudhuffushi,MV,81.52,6.6221,73.0700,
24,Acapulco,MX,87.62,16.8634,-99.8901,


In [25]:
# Set parameters to search for a hotel.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key,
    'location': "48.8566, 2.3522"
}

# Iterate through the dataframe and find coordinates
for index, row in hotel_df.iterrows():
    # Get the latitude and longitude
    
    lat=row['Lat']
    lng=row['Lng']
    
    # Add the latitude and longitude to the location key for the 'params' dictionary
    params['location'] = f'{lat},{lng}'
    
    # Use the search term 'lodging' and our latitudes and longitudes
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    # Make the request and get the JSON data out of the search
    hotels=requests.get(base_url,params=params).json()
    
    # Grab the first hotel from the results and store its name.
    # Grab the first hotel from the results and store the name.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
        print('Hotel Found')
    except (IndexError):
        print("Hotel not found... skipping.")

Hotel Found
Hotel Found
Hotel not found... skipping.
Hotel Found
Hotel not found... skipping.
Hotel Found
Hotel Found
Hotel Found
Hotel Found
Hotel Found
Hotel Found
Hotel Found
Hotel Found
Hotel Found
Hotel Found
Hotel not found... skipping.
Hotel Found
Hotel Found
Hotel Found
Hotel Found
Hotel Found
Hotel Found
Hotel Found
Hotel Found
Hotel not found... skipping.
Hotel Found
Hotel not found... skipping.
Hotel Found
Hotel Found
Hotel Found
Hotel not found... skipping.
Hotel Found
Hotel Found
Hotel Found
Hotel Found
Hotel Found
Hotel Found
Hotel Found
Hotel Found
Hotel Found
Hotel Found
Hotel not found... skipping.
Hotel Found
Hotel Found
Hotel Found
Hotel Found
Hotel Found
Hotel Found
Hotel Found
Hotel Found
Hotel Found
Hotel Found
Hotel Found
Hotel Found
Hotel Found
Hotel Found
Hotel Found
Hotel Found
Hotel not found... skipping.
Hotel Found
Hotel Found
Hotel Found
Hotel Found
Hotel Found
Hotel Found
Hotel Found
Hotel Found
Hotel Found
Hotel Found
Hotel Found
Hotel Found
Hotel Found


In [26]:
hotel_df.head(10)

,City,Country,Max Temp,Lat,Lng,Hotel Name
0,Yarmouth,US,80.15,41.7057,-70.2286,Blue Rock Resort
1,Bayeux,BR,79.43,-7.1250,-34.9322,Infinity Motel
4,Lodja,CD,78.80,-3.4833,23.4333,
6,Cayenne,GF,82.44,4.9333,-52.3333,Hôtel Le Dronmi
7,Avera,US,80.65,33.1940,-82.5271,
10,Rizhao,CN,76.15,35.4275,119.4553,Novotel Rizhao Suning
14,Kavaratti,IN,80.89,10.5669,72.6420,Green leaf
15,Hilo,US,82.27,19.7297,-155.0900,Hilo Hawaiian Hotel
22,Kulhudhuffushi,MV,81.52,6.6221,73.0700,Haajy Guest House 1
24,Acapulco,MX,87.62,16.8634,-99.8901,HS Hotsson Smart Acapulco


In [28]:
# Add a heatmap of temperature for the vacation spots and marker for each city.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,
             dissipating=False, max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [29]:
info_box_template="""
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

In [30]:
# Store the DataFrame Row.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

In [31]:
# Add a heatmap of temperature for the vacation spots and a pop-up market for each city.
locations = hotel_df[['Lat','Lng']]
max_temp=hotel_df['Max Temp']
fig=gmaps.figure(center=(30.0,31.0),zoom_level=1.5)
heat_layer=gmaps.heatmap_layer(locations,weights=max_temp,dissipating=False,max_intensity=300,point_radius=4)
marker_layer=gmaps.marker_layer(locations,info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# Call the figure to plot the data
fig

Figure(layout=FigureLayout(height='420px'))